## Leetcode SQL Problems - Free Medium

### 177. Nth Highest Salary

In [ ]:
CREATE FUNCTION getNthHighestSalary(N INT) RETURNS INT
BEGIN
  
  DECLARE num int;
  SET num = N-1;
  
  RETURN(
     SELECT DISTINCT Salary
     FROM Employee
     ORDER BY Salary DESC
     LIMIT num,1
  );
  
END

### 178. Rank Scores

In [ ]:
# Solution 1

SELECT 
    s1.Score AS score, 
    (
        SELECT COUNT(DISTINCT s2.Score)
        FROM Scores s2
        WHERE s1.Score <= s2.Score
    ) AS 'Rank'
FROM Scores s1
ORDER BY s1.Score DESC

In [ ]:
# Solution 2

SELECT
    Score AS score,
    DENSE_RANK() OVER(ORDER BY Score DESC) AS 'Rank'
FROM Scores
ORDER BY Score DESC

### 180. Consecutive Numbers

In [ ]:
# Solution 1
SELECT DISTINCT num AS ConsecutiveNums
FROM Logs l1
WHERE 
    l1.Num = (
        SELECT Num
        FROM Logs l2
        WHERE l2.Id - 1 = l1.Id
    )
    AND l1.Num = (
        SELECT Num
        FROM Logs l3
        WHERE l3.Id - 2 = l1.Id
    )

In [ ]:
# Solution 2
SELECT DISTINCT l1.Num AS ConsecutiveNums
FROM
    Logs l1,
    Logs l2,
    Logs l3
WHERE
    l1.Id = l2.Id - 1
    AND l1.Id = l3.Id - 2
    AND l1.Num = l2.Num
    AND l1.Num = l3.Num

### 184. Department Highest Salary

In [ ]:
# Solution 1
WITH depart_name AS (
    SELECT 
        e.*,
        d.Name AS Department 
    FROM Employee e
    LEFT JOIN Department d
    ON e.DepartmentId = d.Id
)

SELECT 
    d.Department,
    d.Name AS Employee,
    d.Salary
FROM 
    depart_name d,
    (SELECT 
        Department,
        MAX(Salary) AS max_s
    FROM depart_name
    GROUP BY Department) m
WHERE d.Department = m.Department
AND d.Salary = m.max_s


In [ ]:
# Solution 2
SELECT 
    d.Name AS Department,
    e.Name AS Employee,
    e.Salary
FROM 
    Employee e
    JOIN Department d
    ON e.DepartmentId = d.Id
WHERE (DepartmentId, Salary) IN 
    (SELECT 
        DepartmentId,
        MAX(Salary)
    FROM Employee
    GROUP BY DepartmentId)

### 626. Exchange Seats

In [ ]:
# Solution 1
SELECT 
    s1.id, (
        SELECT CASE
            WHEN 
            MOD(s1.id, 2) = 1 
            AND s1.id = (SELECT COUNT(*) FROM seat s1)
            THEN (SELECT student FROM seat s2 WHERE s2.id = s1.id)

            WHEN 
                MOD(s1.id, 2) = 1 
                AND s1.id <> (SELECT COUNT(*) FROM seat s1)
            THEN (SELECT student FROM seat s2 WHERE s2.id = s1.id + 1)

            WHEN MOD(s1.id, 2) = 0
            THEN (SELECT student FROM seat s2 WHERE s2.id = s1.id - 1)        
        END
    ) AS 'student'
FROM  seat s1
ORDER BY s1.id


In [ ]:
# Solution 2
WITH counts AS (SELECT COUNT(*) AS count FROM seat)

SELECT 
    (SELECT CASE
        WHEN MOD(id, 2) = 1 AND id = counts.count
        THEN id
        WHEN MOD(id, 2) = 1 AND id <> counts.count
        THEN id + 1
        ELSE id - 1       
    END) AS id,
    student
FROM seat, counts
ORDER BY id